Detailed Editor
  1. User needs to set userkey, adjust database host if necessary
  2. run the first two cells, and then choose the event to be edited, reviewed
  3. run the following cell to load that event's data
      1. ( if desired after editing properties you may use the "save threat event" button )
  4. run the next cell to see a graphical representation of the distributions
  5. run the last cell to see a simulated annual lost expectancy
  
This editor allows the properties associated with a threat scenario to be modeled with probability distributions

In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
import psycopg2
import qual
import quant


conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)


cursor.execute ('SELECT threat_id, source_id, impact_id, name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init, vdist, vparam2, pdist, pparam2, cap_dist, cap_param2, init_dist, init_param2, impact_dist, impact_param2 FROM detailv')
triages = cursor.fetchall()

rec = {}

def update_row():
    print ("saving threat event ")
    row = rec['row']
    w = rec['w']
    vuln_val = w[5].value
    vdist = quant.prob_dict[w[14].value]
    pdist = quant.prob_dict[w[16].value]
    idist = quant.prob_dict[w[18].value]
    cdist = quant.prob_dict[w[20].value]
    ndist = quant.prob_dict[w[22].value]
    perv_val = w[6].value
    imp_val = w[7].value
    cap_val = w[12].value
    init_val = w[13].value
    cursor.execute (quant.update_triage_stmt, [w[3].value, w[4].value, vuln_val, vdist, w[15].value, 
                                         perv_val, pdist, w[17].value, imp_val, w[8].value, w[9].value, idist, w[19].value,
                                         w[10].value, w[11].value, cap_val, cdist, w[21].value,
                                         init_val, ndist, w[23].value, row[0], row[1], row[2]])
    conn.commit()
    

# build up a list of the available events
options_list = []
options_mapping = {}


for x in range(0, len(triages)):
    options_list.append (triages[x][3])
    options_mapping[triages[x][3]] = x

# now create a widget
select_widget = widgets.Dropdown (options=options_list, value=options_list[0], description='Threat Events:')
display (select_widget)



Use the selection above to choose a Threat Event to edit and then run the cell below to edit it.

In [ ]:
x = options_mapping[select_widget.value]

w = [0,0,0]
w.append ( widgets.Text( value=triages[x][3], description='Threat Event:' ))
w.append ( widgets.Textarea( value=triages[x][4], description='Description:' ))
w.append ( widgets.Text( value=str(triages[x][5]), description='Vulnerability' ))
w.append ( widgets.Text( value=str(triages[x][6]), description='Pervasiveness' ))
w.append ( widgets.Text( value=str(triages[x][7]), description='SLE cost' ))
w.append ( widgets.Text( value=str(triages[x][8]), description='Impact:' ))
w.append ( widgets.Textarea( value=triages[x][9], description='Impact Description:' ))
w.append ( widgets.Text( value=triages[x][10], description='Threat Source:' ))
w.append ( widgets.Textarea( value=triages[x][11], description='Source Description:' ))
w.append ( widgets.Text( value=str(triages[x][12]), description='Threat Capacity:' ))
w.append ( widgets.Text( value=str(triages[x][13]), description='10 year num attacks:' ))
w.append ( widgets.Dropdown( options=quant.prob_list, value=quant.prob_rdict[triages[x][14]], description='Distribution:' ))
w.append ( widgets.Text( value=str(triages[x][15]), description='parameter:' ))
w.append ( widgets.Dropdown( options=quant.prob_list, value=quant.prob_rdict[triages[x][16]], description='Distribution:' ))
w.append ( widgets.Text( value=str(triages[x][17]), description='parameter:' ))
w.append ( widgets.Dropdown( options=quant.prob_list, value=quant.prob_rdict[triages[x][18]], description='Distribution:' ))
w.append ( widgets.Text( value=str(triages[x][19]), description='parameter:' ))
w.append ( widgets.Dropdown( options=quant.prob_list, value=quant.prob_rdict[triages[x][20]], description='Distribution:' ))
w.append ( widgets.Text( value=str(triages[x][21]), description='parameter:' ))
w.append ( widgets.Dropdown( options=quant.prob_list, value=quant.prob_rdict[triages[x][22]], description='Distribution:' ))
w.append ( widgets.Text( value=str(triages[x][23]), description='parameter:' ))

rec['row'] = triages[x]
rec['w'] = w 
b = widgets.Button (description = 'save threat event')
    
    
def make_fn ():
    return lambda b: update_row()
         
b.on_click(make_fn())
    
display (widgets.VBox ([
        widgets.HBox([w[3], w[4]]),
        widgets.HBox([w[5], w[14], w[15]]),
        widgets.HBox([w[6], w[16], w[17]]),
        widgets.HBox([w[7], w[18], w[19]]),
        widgets.HBox([w[8], w[9]]),
        widgets.HBox([w[10], w[11]]),
        widgets.HBox([w[12], w[20], w[21]]),
        widgets.HBox([w[13], w[22], w[23]]),
        b,
        widgets.Label ()
    ]))

The idea of a successful attack is that 
  - for each day estimate how many attacks occurred
  - then figure out if the attack was successful based on the source capacity and the vulnerability
  - then estimate the amount of damage

In [ ]:
sim_runs = 1000

event_loss = int(w[7].value)
loss_var = int(w[23].value)
loss_fn = quant.prob_fn[w[22].value]

af = int(w[13].value)
attack_freq = af / 10
source_cap = int(w[12].value)
source_var = int(w[21].value)
cap_fn = quant.prob_fn[w[20].value]

vuln_score = int(w[5].value)
vuln_var = int(w[15].value)
vuln_fn = quant.prob_fn[w[14].value]


attack_chance = quant.prob_fn[w[22].value](attack_freq, int(w[23].value), sim_runs)
cap_chance = cap_fn(source_cap, source_var, sim_runs)
vuln_chance = vuln_fn(vuln_score, vuln_var, sim_runs)

count, bins, ignored = plt.hist(vuln_chance, 30, density=True)
plt.plot ([vuln_score, vuln_score], [0, 0.02], linewidth=2, color='r')
plt.title ("Vulnerability")
plt.show()

data = plt.hist(cap_chance, 30, density=True)
plt.plot ([source_cap, source_cap], [0, 0.02], linewidth=2, color='r')
plt.title("Source Capacity")
plt.show()

data = plt.hist(attack_chance, 30, density=True)
plt.title(str(attack_freq) + " attacks expected annually")
plt.show()



Now iterate through the results for the number of attacks per year and simulate whether each attack is successful or not

In [ ]:
loss_list = []
loss_sum = 0
loss_max = 0

for ys in attack_chance:
    if ys > 0:
        loss = 0
        count = int (ys)
        c = cap_fn (source_cap, source_var, count)
        v = vuln_fn (vuln_score, vuln_var, count)
        l = loss_fn (event_loss, loss_var, count)
        for i in range(1, count):
            if (c[i] >= v[i] ):
                loss = loss + l[i]
        loss_list.append (loss)
        if loss > loss_max:
            loss_max = loss
        loss_sum += loss
     
plt.hist(loss_list, 100, density=True)
plt.title ("Expected annual loss")
plt.show()

print ("max loss is " + str(int(loss_max)))
print ("average loss is " + str(int(loss_sum/len(loss_list))))